# NEXUS tool: case study for the Souss-Massa basin - energy demand calculations
In this notebook a case study for the Souss-Massa basin is covered using the `nexus_tool` package. The water requirements for agricultural irrigation, residential, industrial and tourism use were previously calculated using the Water Evaluation and Planning System (WEAP) model. In this case study, the energy requirements for groundwater pumping, wastewater treatment, desalination of seawater and pumping for water conveyance are estimated.

First import the package by running the following block:

In [1]:
%load_ext autoreload

In [2]:
%autoreload
import os
import nexus_tool
from nexus_tool.weap_tools import create_folder
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [3]:
scenarios = ['Reference']
load_folder = os.path.join('Data - Souss-Massa', 'Merged data')
results_folder = os.path.join('Morocco dashboard', 'data')
for scenario in scenarios:
    scenario_folder = os.path.join(load_folder, scenario)
    create_folder(os.path.join(results_folder, scenario))
    for sub_scenario in os.listdir(scenario_folder):
        sub_scenario_folder = os.path.join(scenario_folder, sub_scenario)
        create_folder(os.path.join(results_folder, scenario, sub_scenario))
        for level in os.listdir(sub_scenario_folder):
            output_folder = os.path.join(results_folder, scenario, sub_scenario, level)
            load_data = os.path.join(sub_scenario_folder, level)
            create_folder(output_folder)
    
            file_path = os.path.join(load_data, 'demand_data.csv')
            df = nexus_tool.read_csv(file_path)

            souss_massa = nexus_tool.Model(df)

            souss_massa.df['Pipe_diameter'] = 0.4
            souss_massa.elevation = 'wtd'
            souss_massa.L = 'wtd'
            souss_massa.peak_Q = 'value'
            souss_massa.avg_Q = 'value'

            souss_massa.get_A(inplace=True)
            souss_massa.get_V(inplace=True, axis=0)
            souss_massa.get_Re(inplace=True, axis=0)
            souss_massa.get_f(inplace=True, axis=0)

            souss_massa.get_sw_tdh(inplace = True, axis=0)
            souss_massa.get_SWpumping_energy(inplace = True, axis=0)

            file_path = os.path.join(load_data, 'wwtp_inflow.csv')
            df_wwtp = pd.read_csv(file_path)

            wwtp_energy_int = 0.6 # kWh/m3
            df_wwtp['SWPA_E_'] = df_wwtp.value * wwtp_energy_int
            
            souss_massa.end_year = 2050
            souss_massa.start_year = 2020
            souss_massa.discount_rate = 0.05
            
            souss_massa.create_wind_turbine('Wind power', life=20,
                                            om_cost=0.02, capital_cost=1300,
                                            efficiency=0.6)
            
            souss_massa.create_pv_system('Solar PV', life=15,
                                         om_cost=0.01, capital_cost=1140,
                                         efficiency=0.7)
            
            souss_massa.create_standard_tech('Diesel set', life=10, om_cost=0.1,
                                             capital_cost=938, fuel_cost=0.6,
                                             fuel_req=0.095, efficiency=0.27, cf = 0.5,
                                             emission_factor=2.7, env_cost=0)
            
            souss_massa.df['wind'] = np.random.randint(5,15,souss_massa.df.shape[0])
            souss_massa.df['srad'] = np.random.randint(2000,3000,souss_massa.df.shape[0])
            souss_massa.get_cf('all', axis=0)
#             jordan_gw.df.loc[jordan_gw.df.Year>=2020].to_csv(os.path.join(output_folder, 'Groundwater_pumping.csv'), index=False)
#             df_wwtp.loc[df_wwtp.Year>=2020].to_csv(os.path.join(output_folder, 'wwtp_data.csv'), index=False)

In [4]:
# souss_massa.technologies['Wind power'].cf
# souss_massa.technologies['Solar PV'].cf
souss_massa.technologies['Diesel set'].cf

0.5

In [5]:
souss_massa.df

,Year,Month,links,value,Demand point,Supply point,type,wtd,Pipe_diameter,Pipe_area,Flow_velocity_,Re_,f_,tdh_sw,SWPP_E_,SWPA_E_,wind,srad
0,1976,9,Transmission Link from Desalination to Agadir,0.000,Agadir,Desalination,Desalination plant,NaN,0.4,0.125664,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,11,2434
1,1976,10,Transmission Link from Desalination to Agadir,0.000,Agadir,Desalination,Desalination plant,NaN,0.4,0.125664,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,13,2145
2,1976,11,Transmission Link from Desalination to Agadir,0.000,Agadir,Desalination,Desalination plant,NaN,0.4,0.125664,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,9,2065
3,1976,12,Transmission Link from Desalination to Agadir,0.000,Agadir,Desalination,Desalination plant,NaN,0.4,0.125664,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,9,2430
4,1977,1,Transmission Link from Desalination to Agadir,0.000,Agadir,Desalination,Desalination plant,NaN,0.4,0.125664,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,7,2052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17479,2007,4,Transmission Link from Youssef Ben Tachefine t...,654.030,Tiznit,Youssef Ben Tachefine,Domestic,NaN,0.4,0.125664,0.004819,1.927632e-06,0.001353,NaN,NaN,NaN,9,2886
17480,2007,5,Transmission Link from Youssef Ben Tachefine t...,187.153,Tiznit,Youssef Ben Tachefine,Domestic,NaN,0.4,0.125664,0.001379,5.515986e-07,0.001154,NaN,NaN,NaN,6,2499
17481,2007,6,Transmission Link from Youssef Ben Tachefine t...,0.000,Tiznit,Youssef Ben Tachefine,Domestic,NaN,0.4,0.125664,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,14,2041
17482,2007,7,Transmission Link from Youssef Ben Tachefine t...,0.000,Tiznit,Youssef Ben Tachefine,Domestic,NaN,0.4,0.125664,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,9,2983


In [6]:
water = souss_massa.df.loc[(souss_massa.df.Year==2007)].value.sum()

In [7]:
energy = souss_massa.df.loc[(souss_massa.df.Year==2007)].SWPA_E_.sum()

In [8]:
energy/water

0.17642552892565144